# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning

# Creación de Dataframes

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

En este notebook, que es un re-trabajo sobre el `Notebook Anexo` del [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) se buscan features del set de datos y se crean nuevos atributos a partir de ellos. Estos se almacenan en nuevos archivos `.csv` para luego ser concatenados al set de datos principal.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

df = pd.read_csv('./data/events_up_to_01062018.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.head()

## brands.csv

Marcas de celulares

In [ ]:
df['model'].unique()

In [ ]:
checked = ['iphone', 'samsung', 'motorola', 'lenovo', 'sony', 'lg', 'ipad', 'asus', 'quantum', 'blackberry']
model_parsed = df['model'].dropna().map(lambda x: x.lower())
model_parsed = model_parsed.map(lambda x: x.split())

def find_brand(model):
    for str in model:
        if str in checked:
            return str
    return "other"


df['brand'] = model_parsed.map(find_brand)
df['brand'] = df['brand'].astype('category')

In [ ]:
df[['model', 'brand']].dropna().drop_duplicates().to_csv('data/brands.csv', index=False)

In [ ]:
brands_csv = pd.read_csv("data/brands.csv", low_memory=False)
display(brands_csv.head())
display(brands_csv.shape)

## os.csv

Sistemas operativos de celulares

In [ ]:
df['operating_system_version'].unique()

In [ ]:
checked = ['windows', 'android', 'linux', 'mac', 'ios', 'ubuntu', 'chrome os', 'tizen', 'blackberry', 'other']
os_version_parsed = df['operating_system_version'].dropna().map(lambda x: x.lower())

def find_os(os_version):
    for os in checked:
        if os in os_version:
            return os
    return "another"


df['operating_system'] = os_version_parsed.map(find_os)
df['operating_system'] = df['operating_system'].astype('category')

In [ ]:
# Chequeamos cuantos os quedaron con el nombre 'another' (idealmente, ninguno)
df[['operating_system_version', 'operating_system']].dropna().head(10)
df[df['operating_system'] == 'another'][['operating_system_version', 'operating_system']].head(10)

In [ ]:
df['operating_system'].value_counts()

In [ ]:
df[['operating_system_version', 'operating_system']].dropna().drop_duplicates().to_csv('data/os.csv', index=False)

In [ ]:
os_csv = pd.read_csv("data/os.csv", low_memory=False)
display(os_csv.head())
display(os_csv.shape)

## browsers.csv

Exploradores de internet desde los cuales se accedió al sitio

In [ ]:
df['browser_version'].unique().tolist()

In [ ]:
checked = ['mobile safari', 'chrome mobile', 'ie mobile', 'firefox mobile', 'edge mobile', 'opera mobile',
           'mobile', 'blackberry os', 'blackberry webkit', 'chrome', 'android', 'opera', 'ie', 'firefox', 
           'facebook', 'samsung', 'chromium', 'edge', 'yandex', 'uc', 
           'other', 'safari', 'puffin', 'maxthon', 'vivaldi', 'pinterest']
browser_version_parsed = df['browser_version'].dropna().map(lambda x: x.lower())

def find_browser(browser_version):
    for browser in checked:
        if browser in browser_version:
            return browser
    return "other"


df['browser'] = browser_version_parsed.map(find_browser)
df['browser'] = df['browser'].astype('category')

In [ ]:
df[['browser_version', 'browser']].dropna()
display(df['browser'].value_counts().head())
df[['browser_version', 'browser']].head()
df[df['browser'] == 'other']['browser_version']

In [ ]:
df[['browser_version', 'browser']].dropna().drop_duplicates().to_csv('data/browsers.csv', index=False)

In [ ]:
browsers_csv = pd.read_csv("data/browsers.csv", low_memory=False)
display(browsers_csv.head())
display(browsers_csv.shape)

## sessions.csv

Eventos por usuario

Definimos una sesión como una serie de eventos por usuario, los cuales están todos con menos de 30 minutos de inactividad entre el actual y el anterior.

In [ ]:
funnel = df
funnel = funnel.sort_values(['person', 'timestamp'])
funnel.head()

In [ ]:
funnel['time_diff'] = funnel.groupby('person')['timestamp'].diff()
funnel['time_diff'] = funnel['time_diff'].fillna(0)
funnel['time_diff_min'] = funnel['time_diff'] / np.timedelta64(1, 'm')

In [ ]:
funnel[['person', 'event', 'timestamp', 'time_diff', 'time_diff_min']].head()

In [ ]:
THRESHOLD = 30 # minutos

funnel['new_session'] = funnel['time_diff_min'] > THRESHOLD
funnel['session_id'] = funnel.groupby('person')['new_session'].cumsum()
funnel['session_id'] = funnel['session_id'].astype('int')

In [ ]:
funnel[['person', 'new_session', 'session_id']].head()

In [ ]:
gb = funnel.groupby(['person', 'session_id'])

funnel['session_cumno'] = gb.cumcount()
funnel['session_total_events'] = gb['session_cumno'].transform(lambda x: x.size)
funnel['session_first'] = funnel['session_cumno'] == 0
funnel['session_last'] = funnel['session_cumno'] == (-1+funnel['session_total_events'])

In [ ]:
cols = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'event', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last']

funnel[cols].head()

In [ ]:
funnel['is_conversion'] = funnel['event'] == 'conversion'
gb = funnel.groupby(['person', 'session_id'])
funnel['session_total_conversions'] = gb['is_conversion'].transform(lambda x: x.sum())
funnel['session_has_conversion'] = funnel['session_total_conversions'] > 0

funnel.head()

In [ ]:
funnel['is_checkout'] = funnel['event'] == 'checkout'
gb = funnel.groupby(['person', 'session_id'])
funnel['session_total_checkouts'] = gb['is_checkout'].transform(lambda x: x.sum())
funnel['session_has_checkout'] = funnel['session_total_checkouts'] > 0

funnel.head()

In [ ]:
funnel['ad_origin'] = (funnel['event'] == 'ad campaign hit') & funnel['session_first']
gb = funnel.groupby(['person', 'session_id'])
funnel['session_ad'] = gb['ad_origin'].transform(lambda x: x.sum())
funnel['session_ad'] = funnel['session_ad'] > 0

funnel.head()

In [ ]:
funnel[cols].head()

In [ ]:
cols_csv = ['time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last', \
        'session_total_conversions', 'session_has_conversion', \
        'session_total_checkouts', 'session_has_checkout', 'session_ad']

funnel = funnel.sort_index()

funnel[cols].head()

In [ ]:
cols_check_ord = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last', \
        'session_total_conversions', 'session_has_conversion', \
        'session_total_checkouts', 'session_has_checkout', 'session_ad']

funnel.sort_values(['person', 'timestamp'])[cols_check_ord].head()

In [ ]:
funnel[cols_csv].to_csv('data/sessions.csv', index=False)

In [ ]:
sessions_csv = pd.read_csv("data/sessions.csv", low_memory=False)
display(sessions_csv.head())
display(sessions_csv.shape)

## prices.csv

In [ ]:
mapping_conditions = {'Bom':15,
                      'Muito Bom': 6,
                      'Excelente': 5,
                      'Bom - Sem Touch ID':17,
                      'Novo':18,
                       np.nan:0
                     }

df_tmp = df.loc[df['model'].notnull()][['sku','model','color','storage','condition']]
df_tmp['to_search'] = df_tmp['model'] + ' ' + df_tmp['storage'].astype(str) + ' ' + df_tmp['color']
df_tmp['condition_n'] = df_tmp['condition'].transform(lambda x: mapping_conditions[x])
df_tmp['to_search+condition_n'] = df_tmp['to_search'] + ' ' + df_tmp['condition_n'].astype(str)

df_tmp = df_tmp.drop_duplicates()
df_tmp = df_tmp.dropna()
display(df_tmp.head())

dic_modelos_condiciones = {}
modelos = df_tmp['to_search'].tolist()
condiciones = df_tmp['condition_n'].tolist()
for m,c in zip(modelos,condiciones):
    if m in dic_modelos_condiciones:
        dic_modelos_condiciones[m].append(c)
    else: dic_modelos_condiciones[m] = [c]


In [ ]:
def request_soup(url):
    try:
        req = requests.get(url)
        html = req.content
        soup = BeautifulSoup(html, "lxml")
    except requests.exceptions.ConnectionError:
        raise ConnectionError(f"Connection with {url} refused.")
    return soup

BASE = 'https://www.trocafone.com/comprar/list?'
MODELO = 'q'
CONDICION = 'condition'

precios = {}
i=1
for modelo, condiciones in dic_modelos_condiciones.items(): 
    print(f'Iteración {i} de {len(dic_modelos_condiciones)}:')
    modelo_s = modelo.replace(' ','+')
    for condicion in condiciones:
        url = f'{BASE}{MODELO}={modelo_s}&{CONDICION}={condicion}'
        print(f'\t * {url}: ',end='')
        soup = request_soup(url)
        precio = soup.find(class_='price-value')
        if precio and precio.string:
            precio = precio.string
        else: precio = 0
        print(f'${precio}')
        precios[f'{modelo} {condicion}'] = precio
    i+=1

In [ ]:
orden = df_tmp['to_search+condition_n']

precios_ordenado = []

for modelo in orden:
    precios_ordenado.append(precios[modelo])

df_tmp['precio_reales'] = precios_ordenado

In [ ]:
df_tmp[['sku','precio_reales']].to_csv('data/prices.csv', index=False)


In [ ]:
prices_csv = pd.read_csv("data/prices.csv", low_memory=False)
display(prices_csv.head())
display(prices_csv.shape)